In [1]:
# Importing the patstat client
from epo.tipdata.patstat import PatstatClient

# Initialize the PATSTAT client
patstat = PatstatClient()

# Access ORM
db = patstat.orm()

In [8]:
# Importing tables as models
from epo.tipdata.patstat.database.models import TLS201_APPLN, TLS207_PERS_APPLN, TLS206_PERSON

# Define the query in ORM
q = db.query(
    TLS201_APPLN.appln_id,
    TLS201_APPLN.appln_nr,
    TLS201_APPLN.appln_filing_date,
    TLS201_APPLN.nb_citing_docdb_fam,  # number of families citing the application
    TLS206_PERSON.person_name  # inventor's name
).join(
    TLS207_PERS_APPLN, TLS201_APPLN.appln_id == TLS207_PERS_APPLN.appln_id
).join(
    TLS206_PERSON, TLS207_PERS_APPLN.person_id == TLS206_PERSON.person_id
).filter(
    TLS201_APPLN.appln_filing_year >= 2020,
    TLS201_APPLN.appln_auth == 'EP',
    TLS201_APPLN.granted == 'Y',
    TLS207_PERS_APPLN.invt_seq_nr > 0  # filter to include only inventors
)

# Creating a dataframe with the results
res = patstat.df(q)

# Sorting the results by application ID
res_sorted = res.sort_values(by='person_name', ascending=False)

res_sorted


,appln_id,appln_nr,appln_filing_date,nb_citing_docdb_fam,person_name
9273,524815902,20151377,2020-01-13,0,"Ünker, Yavuz"
12209,528859167,20714139,2020-03-11,0,"Übermuth, Tim"
49776,524947511,20152090,2020-01-16,1,"Übelacker, Roland"
49777,524947529,20152100,2020-01-16,1,"Übelacker, Roland"
27941,527050575,20159886,2020-02-27,0,"ÜYÜCÜ, Metehan"
...,...,...,...,...,...
21750,526086085,20156535,2020-02-11,0,
21749,544600154,21152589,2021-01-20,0,
21748,525924233,20156103,2020-02-07,0,
21747,527335300,20161299,2020-03-05,0,


In [9]:
# Importing necessary modules
from sqlalchemy import func
from epo.tipdata.patstat.database.models import TLS201_APPLN, TLS207_PERS_APPLN, TLS206_PERSON

# Define the query in ORM
q = db.query(
    TLS206_PERSON.person_name,  # inventor's name
    func.sum(TLS201_APPLN.nb_citing_docdb_fam).label('total_citations')  # sum of citing families
).join(
    TLS207_PERS_APPLN, TLS201_APPLN.appln_id == TLS207_PERS_APPLN.appln_id
).join(
    TLS206_PERSON, TLS207_PERS_APPLN.person_id == TLS206_PERSON.person_id
).filter(
    TLS201_APPLN.appln_filing_year >= 2020,
    TLS201_APPLN.appln_auth == 'EP',
    TLS201_APPLN.granted == 'Y',
    TLS207_PERS_APPLN.invt_seq_nr > 0  # filter to include only inventors
).group_by(
    TLS206_PERSON.person_name
)

# Creating a dataframe with the results
res = patstat.df(q)

# Sorting the results by total citations in descending order
res_sorted = res.sort_values(by='total_citations', ascending=False)

res_sorted


,person_name,total_citations
177,"HARRIS, Jason L.",1013
203,"BOUDREAUX, Chad P.",604
1093,"TIMM, Richard W.",604
73,"BAKOS, Gregory J.",604
976,"SHELTON, IV, Frederick E.",522
...,...,...
28470,"PAMULAPARTHY, Venkata Dhruva",0
28471,"PING, Er Xuan",0
28472,"Commin, Paul",0
28473,"GRUSS, Jean-Antoine",0
